# Fine-Tuning Language Models from Human Preferences

Обзор работы по по использованию асесорской ифнормации для дообучения модели на основе обучения с подкреплением. [Статья по ссылке](https://arxiv.org/pdf/1909.08593.pdf).

## Библиотеки

In [1]:
import io
import math
import time
from tqdm import tqdm
from collections import Counter

import torch
import numpy as np
import scipy.spatial
import torch.nn as nn
from torch import Tensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)

import torchtext
from torchtext.vocab import Vocab
from torchtext.data.utils import get_tokenizer
from torchtext.utils import download_from_url, extract_archive

from transformers import AutoModel, AutoTokenizer

In [2]:
# Убирем рандом
_ = torch.manual_seed(0)

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

# Базовый пример обучения генерации парафраза

В работе рассматривается задача сумаризации, а это немного сложно... рассмотри их подход для задачи генерации парафраза... это немного проще...

### Скачиваем данные для парафраза на английском.
[Статья 2021 года. ParaSCI: A Large Scientific Paraphrase Dataset for Longer Paraphrase Generation.](https://github.com/dqxiu/ParaSCI)

In [4]:
url_base = 'https://raw.githubusercontent.com/dqxiu/ParaSCI/master/Data/ParaSCI-ACL/'
train_urls = ('train/train.src', 'train/train.tgt')
val_urls = ('val/val.src', 'val/val.tgt')
test_urls = ('test/test.src', 'test/test.tgt')

train_filepaths = [download_from_url(url_base + url) for url in train_urls]
val_filepaths = [download_from_url(url_base + url) for url in val_urls]
test_filepaths = [download_from_url(url_base + url) for url in test_urls]

### Строим словарь

In [5]:
tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

def build_vocab(filepaths, tokenizer):
    counter = Counter()
    for filepath in filepaths:
        with io.open(filepath, encoding="utf8") as f:
            for string_ in f:
                counter.update(tokenizer(string_))
    return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

vocab = build_vocab(train_filepaths, tokenizer)

In [6]:
len(vocab)

14821

### Предобработка данных

In [7]:
def data_process(filepaths):
    raw_src_iter = iter(io.open(filepaths[0], encoding="utf8"))
    raw_tgt_iter = iter(io.open(filepaths[1], encoding="utf8"))
    data = []
    for (raw_src, raw_tgt) in zip(raw_src_iter, raw_tgt_iter):
        src_tensor_ = torch.tensor(
            [vocab[token] for token in tokenizer(raw_src.rstrip("\n"))],
            dtype=torch.long)
        tgt_tensor_ = torch.tensor(
            [vocab[token] for token in tokenizer(raw_tgt.rstrip("\n"))],
            dtype=torch.long)
        data.append((src_tensor_, tgt_tensor_))
    return data

train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

### Финальные датасеты для обучения

In [8]:
BATCH_SIZE = 16
PAD_IDX = vocab['<pad>']
BOS_IDX = vocab['<bos>']
EOS_IDX = vocab['<eos>']

In [9]:
def generate_batch(data_batch):
    src_batch, tgt_batch = [], []
    for (src_item, tgt_item) in data_batch:
        src_batch.append(torch.cat([torch.tensor([BOS_IDX]), src_item, torch.tensor([EOS_IDX])], dim=0))
        tgt_batch.append(torch.cat([torch.tensor([BOS_IDX]), tgt_item, torch.tensor([EOS_IDX])], dim=0))
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

### Класс модели SEQ2SEQ transformer

#### Определение самой модели

In [10]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

    
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding +
                            self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)
    
# Делаем, так чтобы в обучении не было заглядывания на дальнешие слова
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

#### Определения декодирования

In [11]:
def greedy_decode(model, src, src_mask, max_len, start_symbol, num_samples=1):
    src = src.to(DEVICE)
    src = torch.cat([src]*num_samples, dim=1)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    
    ys = torch.ones(1, num_samples).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(DEVICE).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.detach()

        ys = torch.cat([ys,
                        next_word.view(1, -1)], dim=0)
    return ys.transpose(0,1)

def sampling_decode(model, src, src_mask, max_len, start_symbol, num_samples=1):
    src = src.to(DEVICE)
    src = torch.cat([src]*num_samples, dim=1)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    
    ys = torch.ones(1, num_samples).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(DEVICE).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        next_word = torch.multinomial(torch.nn.functional.softmax(prob, dim=-1), 1)
        next_word = next_word.detach()

        ys = torch.cat([ys,
                        next_word.view(1, -1)], dim=0)
    return ys.transpose(0,1)

def paraphrase(model, 
              srcs, 
              src_vocab, 
              tgt_vocab, 
              src_tokenizer, 
              decoder=greedy_decode, 
              ret_tokens=False, 
              ret_idx=False, 
              max_len_add=10,
              input_idx=False,
              **argv):
    model.eval()
    global_answers = []
    for src in srcs:
        if not input_idx:
            tokens = [BOS_IDX] + [src_vocab.stoi[tok] for tok in src_tokenizer(src)]+ [EOS_IDX]
            src = torch.LongTensor(tokens)
        num_tokens = len(src)
        src = src.reshape(num_tokens, 1)
        
        src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
        tgt_tokens = decoder(model, src, src_mask, max_len=num_tokens + max_len_add, start_symbol=BOS_IDX, **argv)

        answers = []
        for tgt_token in tgt_tokens:
            if not ret_idx:
                reference = []
                for tok in tgt_token:
                    if tok.item() == tgt_vocab['<eos>']:
                        break
                    if tok.item() not in {tgt_vocab['<eos>'], tgt_vocab['<bos>'], tgt_vocab['<pad>']}:
                        reference.append(tgt_vocab.itos[tok])
                answers.append(" ".join(reference).strip())
                if ret_tokens:
                    answers[-1] = answers[-1].split(" ")
            else:
                reference = []
                for tok in tgt_token:
                    if tok.item() == tgt_vocab['<eos>']:
                        break
                    if tok.item() not in {tgt_vocab['<eos>'], tgt_vocab['<bos>'], tgt_vocab['<pad>']}:
                        reference.append(tok.item())
                        
                answers.append(reference)
        global_answers.append(answers)
    return global_answers

In [12]:
def evaluate(model, val_iter):
    model.eval()
    losses = 0
    for idx, (src, tgt) in (enumerate(valid_iter)):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                              src_padding_mask, tgt_padding_mask, src_padding_mask)
        
        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
    return losses / len(val_iter)

## Базовое обучение без RL

### Функции обучения

In [13]:
def train_epoch(model, train_iter, optimizer, loss_fn):
    model.train()
    losses = 0
    for idx, (src, tgt) in enumerate(train_iter):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, 
                       tgt_input, 
                       src_mask, 
                       tgt_mask,
                       src_padding_mask, 
                       tgt_padding_mask, 
                       src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
    return losses / len(train_iter)

### Инициализация модели

In [14]:
SRC_VOCAB_SIZE = len(vocab)
TGT_VOCAB_SIZE = len(vocab)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 20


transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, 
                                 NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, 
                                 TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)

### Обучение модели

In [15]:
for epoch in range(1, NUM_EPOCHS+1):
    start_time = time.time()
    train_loss = train_epoch(transformer, train_iter, optimizer, loss_fn)
    end_time = time.time()
    val_loss = evaluate(transformer, valid_iter)
    all_time = time.time()
    print(f"Epoch: {epoch}, "
          f"Train loss: {train_loss:.3f}, "
          f"Val loss: {val_loss:.3f}, "
          f"Epoch time = {(end_time - start_time):.3f}s, "
          f"All time = {(all_time - start_time):.3f}s")

Epoch: 1, Train loss: 4.585, Val loss: 2.956, Epoch time = 59.204s, All time = 60.737s
Epoch: 2, Train loss: 3.446, Val loss: 2.353, Epoch time = 61.746s, All time = 63.469s
Epoch: 3, Train loss: 3.006, Val loss: 2.021, Epoch time = 62.291s, All time = 64.029s
Epoch: 4, Train loss: 2.695, Val loss: 1.789, Epoch time = 62.128s, All time = 63.668s
Epoch: 5, Train loss: 2.447, Val loss: 1.607, Epoch time = 60.461s, All time = 62.186s
Epoch: 6, Train loss: 2.238, Val loss: 1.461, Epoch time = 62.401s, All time = 64.144s
Epoch: 7, Train loss: 2.055, Val loss: 1.390, Epoch time = 62.170s, All time = 63.888s
Epoch: 8, Train loss: 1.897, Val loss: 1.301, Epoch time = 60.197s, All time = 61.928s
Epoch: 9, Train loss: 1.755, Val loss: 1.255, Epoch time = 62.284s, All time = 64.039s
Epoch: 10, Train loss: 1.632, Val loss: 1.191, Epoch time = 62.313s, All time = 64.033s
Epoch: 11, Train loss: 1.525, Val loss: 1.164, Epoch time = 60.097s, All time = 61.862s
Epoch: 12, Train loss: 1.425, Val loss: 1

### Пример работы (greedy search)

In [16]:
%%time
paraphrase(transformer, ["in our work , we focus on supervised domain adaptation ."], 
          vocab, 
          vocab, 
          tokenizer, 
          decoder=greedy_decode, num_samples=5)[0]

CPU times: user 97.8 ms, sys: 740 µs, total: 98.5 ms
Wall time: 103 ms


['in our work , we investigate the use of domain adaptation .',
 'in our work , we investigate the use of domain adaptation .',
 'in our work , we investigate the use of domain adaptation .',
 'in our work , we investigate the use of domain adaptation .',
 'in our work , we investigate the use of domain adaptation .']

### Пример работы (multinominal sampling)

In [17]:
%%time
paraphrase(transformer, ["in our work , we focus on supervised domain adaptation ."], 
          vocab, 
          vocab, 
          tokenizer, 
          decoder=sampling_decode, num_samples=5)[0]

CPU times: user 84.2 ms, sys: 22.2 ms, total: 106 ms
Wall time: 105 ms


['in our work , we tackle the task of domain adaptation .',
 'in this work , we focus on semi - supervised approach .',
 'in the inventory presented here , we use a supervised method for domain adaptation .',
 'in our work , we investigate the use of word - level features .',
 'in our work , we propose a method to improve supervised .']

## Обучение с RL
Продолжаем обучение уже используя модель из предыдущего пункта.

В работе предлагается следующий подход.

In [18]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
bert = AutoModel.from_pretrained("sentence-transformers/LaBSE").to(DEVICE)
_ = bert.eval()

Some weights of the model checkpoint at sentence-transformers/LaBSE were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
class Assesor(object):
    def __init__(self):
        pass
        
    def get_best(self, reference, candidates):
        with torch.no_grad():
            tokes = tokenizer(
                [reference], return_tensors='pt', 
                padding=True, max_length=512, truncation=True).to(DEVICE)
            ref_emb = bert(**tokes)[1].cpu().numpy()
            tokes = tokenizer(
                candidates, return_tensors='pt', 
                padding=True, max_length=512, truncation=True).to(DEVICE)
            can_emb = bert(**tokes)[1].cpu().numpy()

        distances = scipy.spatial.distance.cdist(can_emb, 
                                                 ref_emb,
                                                 metric='cosine')
    
        return np.argmin(distances.reshape(-1))
    
class Reward(nn.Module):
    def __init__(self):
        super(Reward, self).__init__()
        self.linear = nn.Linear(1536, 1)
        
    def forward(self, references, candidates):
        assert len(references) == len(candidates)
        with torch.no_grad():
            tokes = tokenizer(
                references, return_tensors='pt', 
                padding=True, max_length=512, truncation=True).to(DEVICE)
            ref_emb = bert(**tokes)[1]
            
            list_of_features = []
            candidates = np.array(candidates)
            for i in range(candidates.shape[1]):
                tokes = tokenizer(
                    candidates[:, i].tolist(), return_tensors='pt', 
                    padding=True, max_length=512, truncation=True).to(DEVICE)
                can_emb = bert(**tokes)[1]

                list_of_features.append(torch.cat([ref_emb, can_emb], dim=-1).unsqueeze(1))
            
            features = torch.cat(list_of_features, dim=1)
        return self.linear(features).squeeze(-1)

In [20]:
assesor = Assesor()

### Генерим выборку для разметки "асессорами" + сразу ее разметим "асессорами"

In [21]:
assessor_dataset = []
for ref in tqdm(train_data[:1000]):
    reference = ' '.join([vocab.itos[item] for item in ref[1]])
    top_cand = ' '.join([vocab.itos[item] for item in ref[0]])
    candidates=paraphrase(
        transformer, [reference], 
        vocab, vocab, tokenizer, decoder=sampling_decode, num_samples=4)[0]
    candidates[np.random.randint(0, 4)] = top_cand
    assessor_dataset.append((reference, candidates, assesor.get_best(reference, candidates)))

100%|██████████| 1000/1000 [01:34<00:00, 10.55it/s]


### Обучаем Reward модель на основе асессорской разметки

In [22]:
def generate_batch_reward(data_batch):
    ref_batch, cand_batch, tgt_batch = [], [], []
    for (ref, candidates, b) in data_batch:
        ref_batch.append(ref)
        cand_batch.append(candidates)
        tgt_batch.append(b)
    return ref_batch, cand_batch, tgt_batch

train_reward_iter = DataLoader(assessor_dataset, batch_size=64,
                               shuffle=True, collate_fn=generate_batch_reward)

In [23]:
def train_epoch_reward(model, train_iter, optimizer, loss_fn):
    model.train()
    losses = 0
    for idx, (ref, cands, tgt) in enumerate(train_iter):
        logits = model(ref, cands)
        
        optimizer.zero_grad()
        
        loss = -1*torch.gather(
            input=torch.nn.functional.log_softmax(logits, dim=-1), 
            dim=1,
            index=torch.tensor(tgt).long().to(DEVICE).view(-1,1)).mean()
        
        loss.backward()

        optimizer.step()
        losses += loss.item()
    return losses / len(train_iter)

In [24]:
reward = Reward()
reward.to(DEVICE)

optimizer = torch.optim.Adam(
    reward.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9, weight_decay=0.5
)

In [25]:
for epoch in range(1, 20):
    start_time = time.time()
    train_loss = train_epoch_reward(reward, train_reward_iter, optimizer, None)
    end_time = time.time()
    print(f"Epoch: {epoch}, "
          f"Train loss: {train_loss:.3f}, "
          f"Epoch time = {(end_time - start_time):.3f}s")

Epoch: 1, Train loss: 1.037, Epoch time = 6.884s
Epoch: 2, Train loss: 0.463, Epoch time = 7.077s
Epoch: 3, Train loss: 0.272, Epoch time = 7.107s
Epoch: 4, Train loss: 0.212, Epoch time = 7.261s
Epoch: 5, Train loss: 0.192, Epoch time = 7.146s
Epoch: 6, Train loss: 0.184, Epoch time = 7.312s
Epoch: 7, Train loss: 0.181, Epoch time = 7.330s
Epoch: 8, Train loss: 0.178, Epoch time = 7.296s
Epoch: 9, Train loss: 0.175, Epoch time = 7.210s
Epoch: 10, Train loss: 0.174, Epoch time = 7.295s
Epoch: 11, Train loss: 0.172, Epoch time = 7.194s
Epoch: 12, Train loss: 0.169, Epoch time = 7.443s
Epoch: 13, Train loss: 0.170, Epoch time = 7.433s
Epoch: 14, Train loss: 0.169, Epoch time = 7.270s
Epoch: 15, Train loss: 0.170, Epoch time = 6.304s
Epoch: 16, Train loss: 0.170, Epoch time = 6.164s
Epoch: 17, Train loss: 0.172, Epoch time = 6.257s
Epoch: 18, Train loss: 0.170, Epoch time = 6.215s
Epoch: 19, Train loss: 0.168, Epoch time = 6.373s


### На основе RL дообучаем модель генерации парафраза

In [26]:
def train_epoch_with_rl(model, train_iter, optimizer, loss_fn, alpha=0.75):
    model.train()
    losses = 0
    for idx, (src, tgt) in enumerate(train_iter):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, 
                       tgt_input, 
                       src_mask, 
                       tgt_mask,
                       src_padding_mask, 
                       tgt_padding_mask, 
                       src_padding_mask)
        
######################RL-start##################################
        logits_batch_first = logits.transpose(0,1)
        # получаем предсказания для RL
        toks = torch.multinomial(
            torch.nn.functional.softmax(
                logits_batch_first.reshape(-1, logits_batch_first.shape[-1]), 
                dim=-1), 
            1).reshape(logits_batch_first.shape[:2])


        references = []
        candidates = []
        for real_toks, pred_toks in zip(tgt[1:,:].transpose(0,1), toks):
            reference = []
            for tok in real_toks:
                if tok.item() == vocab['<eos>']:
                    break
                if tok.item() not in {vocab['<eos>'], vocab['<bos>'], vocab['<pad>']}:
                    reference.append(vocab.itos[tok])
            candidate = []
            for tok in pred_toks:
                if tok.item() == vocab['<eos>']:
                    break
                if tok.item() not in {vocab['<eos>'], vocab['<bos>'], vocab['<pad>']}:
                    candidate.append(vocab.itos[tok])
            references.append(' '.join(reference))
            candidates.append([' '.join(candidate)])

        reward_tr = reward(references, candidates).detach().float().to(DEVICE)

        action_proba = torch.gather(torch.nn.functional.log_softmax(logits_batch_first, dim=-1), 
                                    2, 
                                    toks.view(*logits_batch_first.shape[:2], 1)).squeeze(-1)
######################RL-end###################################
        optimizer.zero_grad()

        loss = alpha*loss_fn(logits.reshape(-1, logits.shape[-1]), tgt[1:,:].reshape(-1)) \
               + (1-alpha)*(-1*reward_tr.view(-1,1)*action_proba).mean()
        
        loss.backward()

        optimizer.step()
        losses += loss.item()
    return losses / len(train_iter)

In [27]:
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)

In [28]:
for epoch in range(1, NUM_EPOCHS+1):
    start_time = time.time()
    train_loss = train_epoch_with_rl(transformer, train_iter, optimizer, loss_fn)
    end_time = time.time()
    val_loss = evaluate(transformer, valid_iter)
    all_time = time.time()
    print(f"Epoch: {epoch}, "
          f"Train loss: {train_loss:.3f}, "
          f"Val loss: {val_loss:.3f}, "
          f"Epoch time = {(end_time - start_time):.3f}s, "
          f"All time = {(all_time - start_time):.3f}s")

Epoch: 1, Train loss: 1.917, Val loss: 1.070, Epoch time = 182.465s, All time = 184.197s
Epoch: 2, Train loss: 2.087, Val loss: 1.092, Epoch time = 178.919s, All time = 180.603s
Epoch: 3, Train loss: 1.794, Val loss: 1.077, Epoch time = 178.733s, All time = 180.430s
Epoch: 4, Train loss: 1.665, Val loss: 1.059, Epoch time = 178.601s, All time = 180.301s
Epoch: 5, Train loss: 1.351, Val loss: 1.064, Epoch time = 180.819s, All time = 182.523s
Epoch: 6, Train loss: 1.229, Val loss: 1.054, Epoch time = 178.216s, All time = 179.920s
Epoch: 7, Train loss: 1.349, Val loss: 1.061, Epoch time = 178.621s, All time = 180.313s
Epoch: 8, Train loss: 1.322, Val loss: 1.054, Epoch time = 178.337s, All time = 180.036s
Epoch: 9, Train loss: 1.516, Val loss: 1.066, Epoch time = 181.325s, All time = 183.020s
Epoch: 10, Train loss: 1.510, Val loss: 1.047, Epoch time = 178.522s, All time = 180.226s
Epoch: 11, Train loss: 1.376, Val loss: 1.071, Epoch time = 177.981s, All time = 179.706s
Epoch: 12, Train lo

### Пример работы (greedy search)

In [29]:
paraphrase(transformer, ["in our work , we focus on supervised domain adaptation ."], 
          vocab, 
          vocab, 
          tokenizer, 
          decoder=greedy_decode, num_samples=5)[0]

['the learning can be viewed as a special case of the .',
 'the learning can be viewed as a special case of the .',
 'the learning can be viewed as a special case of the .',
 'the learning can be viewed as a special case of the .',
 'the learning can be viewed as a special case of the .']

### Пример работы (multinominal sampling)

In [30]:
paraphrase(transformer, ["in our work , we focus on supervised domain adaptation ."], 
          vocab, 
          vocab, 
          tokenizer, 
          decoder=sampling_decode, num_samples=5)[0]

['matching embeddings at 6 first level can be while better suited to latter classes',
 'had 5 distributional .',
 'capacity and anaphoric learning can be adopt to structure generation .',
 'probabilistic labels can be had by aligning translations .',
 'be several generation methods because the morphemes can issues .']